In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s4e7/sample_submission.csv
/kaggle/input/playground-series-s4e7/train.csv
/kaggle/input/playground-series-s4e7/test.csv


In [2]:
train = pd.DataFrame(pd.read_csv("/kaggle/input/playground-series-s4e7/train.csv"))

In [3]:
test = pd.DataFrame(pd.read_csv("/kaggle/input/playground-series-s4e7/test.csv"))

In [4]:
train.head()

,id,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,Response
0,0,Male,21,1,35.0,0,1-2 Year,Yes,65101.0,124.0,187,0
1,1,Male,43,1,28.0,0,> 2 Years,Yes,58911.0,26.0,288,1
2,2,Female,25,1,14.0,1,< 1 Year,No,38043.0,152.0,254,0
3,3,Female,35,1,1.0,0,1-2 Year,Yes,2630.0,156.0,76,0
4,4,Female,36,1,15.0,1,1-2 Year,No,31951.0,152.0,294,0


In [5]:
train['Region_Code'] = train['Region_Code'].astype('int16')
train['Policy_Sales_Channel'] = train['Policy_Sales_Channel'].astype('int16')

In [87]:
df_1 = train[train['Response'] == 1].sample(frac=0.4, random_state=0)
df_0 = train[train['Response'] == 0].sample(frac=0.1, random_state=0)
df = pd.concat([df_1, df_0], axis=0)
print(df_1.describe())
print(df_0.describe())

                 id            Age  Driving_License    Region_Code  \
count  5.660240e+05  566024.000000    566024.000000  566024.000000   
mean   5.758796e+06      43.274147         0.999085      26.857460   
std    3.314025e+06      11.472649         0.030238      11.269714   
min    2.800000e+01      20.000000         0.000000       0.000000   
25%    2.893749e+06      35.000000         1.000000      26.000000   
50%    5.760260e+06      43.000000         1.000000      28.000000   
75%    8.628537e+06      50.000000         1.000000      29.000000   
max    1.150480e+07      82.000000         1.000000      52.000000   

       Previously_Insured  Annual_Premium  Policy_Sales_Channel  \
count       566024.000000   566024.000000         566024.000000   
mean             0.002427    31909.875645             90.362720   
std              0.049209    17651.964555             55.909902   
min              0.000000     2630.000000              1.000000   
25%              0.000000    25978

In [88]:
X = df.drop(['Response', 'id'], axis=1)
y = df['Response']

In [89]:
X.head()

,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage
10628253,Male,39,1,28,0,1-2 Year,Yes,24224.0,26,214
1880466,Male,42,1,28,0,1-2 Year,Yes,44012.0,124,222
5477618,Female,47,1,28,0,1-2 Year,Yes,53511.0,26,115
10390038,Female,38,1,28,0,1-2 Year,Yes,29488.0,26,144
1643786,Male,37,1,7,0,1-2 Year,Yes,30140.0,124,81


In [90]:
cat_features = [0, 2, 3, 4, 5, 6, 8]

In [91]:
from sklearn.model_selection import train_test_split

In [92]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=22)

In [93]:
import gc
# 가비지 컬렉션 수행 전, 객체 수 확인
print("Before GC:", len(gc.get_objects()))

# 가비지 컬렉션 수행
gc.collect()

# 가비지 컬렉션 수행 후, 객체 수 확인
print("After GC:", len(gc.get_objects()))

Before GC: 725305
After GC: 725304


In [94]:
from catboost import CatBoostClassifier
clf = CatBoostClassifier(
    iterations=50,
    learning_rate=0.4,
    depth=8,
    one_hot_max_size=3,
    custom_loss=['AUC', 'Recall']
)

clf.fit(
    X_train, y_train,
    cat_features=cat_features,
    eval_set=(X_val, y_val),
)

0:	learn: 0.5599687	test: 0.5602783	best: 0.5602783 (0)	total: 354ms	remaining: 17.4s
1:	learn: 0.4999781	test: 0.5003656	best: 0.5003656 (1)	total: 733ms	remaining: 17.6s
2:	learn: 0.4670988	test: 0.4676478	best: 0.4676478 (2)	total: 1.08s	remaining: 17s
3:	learn: 0.4480253	test: 0.4485997	best: 0.4485997 (3)	total: 1.47s	remaining: 16.9s
4:	learn: 0.4355138	test: 0.4360560	best: 0.4360560 (4)	total: 1.84s	remaining: 16.6s
5:	learn: 0.4267032	test: 0.4272877	best: 0.4272877 (5)	total: 2.23s	remaining: 16.3s
6:	learn: 0.4193153	test: 0.4197978	best: 0.4197978 (6)	total: 2.6s	remaining: 16s
7:	learn: 0.4157136	test: 0.4162552	best: 0.4162552 (7)	total: 2.96s	remaining: 15.5s
8:	learn: 0.4127441	test: 0.4132545	best: 0.4132545 (8)	total: 3.3s	remaining: 15s
9:	learn: 0.4106492	test: 0.4111360	best: 0.4111360 (9)	total: 3.68s	remaining: 14.7s
10:	learn: 0.4090663	test: 0.4095198	best: 0.4095198 (10)	total: 4.18s	remaining: 14.8s
11:	learn: 0.4074779	test: 0.4079019	best: 0.4079019 (11)	to

In [95]:
y_pred = clf.predict(data=X_val)

In [96]:
def calc_accuracy(y_pred, y_val):
    acc = y_val == y_pred
    i = 0
    for d in acc:
        if d == True:
            i+=1
    s, n = 0, 0
    for j in y_pred:
        n+=1
        if j==1:
            s += 1
    print(i/acc.size, s, n)

In [97]:
calc_accuracy(y_pred, y_val)

0.7884 70059 157500


In [98]:
X_all = train.drop(['Response', 'id'], axis=1)
y_pred_all = clf.predict(data=X_all)

In [85]:
#please execute once
y_all = train['Response']

In [99]:
calc_accuracy(y_pred_all, y_all)

0.7747290304445155 3510234 11504798
